# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, we will train your CNN-RNN model.  

This notebook is divided into following sections:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step we will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch. It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold. Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model (3 recommended). See [This paper](https://arxiv.org/pdf/1502.03044.pdf) for more details.
- `save_every` - determines how often to save the model weights.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.
- `log_file` - Log file containing how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

In [2]:
enableGPU = True

# Utilise GPU if available:
if  enableGPU and torch.cuda.is_available():
    device = torch.device("cuda")
    print('Training on GPU:',torch.cuda.get_device_name())
else:
    device = torch.device("cpu")
    print('Training on CPU ...')

Training on GPU: GeForce GTX 960M


In [3]:
# Select appropriate values for the Python variables below.
batch_size = 96                     # batch size
vocab_threshold = 12                # minimum word count threshold
vocab_from_file = False             # if True, load existing vocab file
embed_size = 384                    # dimensionality of image and word embeddings
hidden_size = 640                   # number of features in hidden state of the RNN decoder
num_epochs = 3                      # number of training epochs
save_every = 1                      # determines frequency of saving model weights
print_every = 100                   # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if device == "cuda" else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

loading annotations into memory...
Done (t=0.99s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=1.32s)
creating index...


  0%|          | 590/414113 [00:00<01:10, 5899.95it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:59<00:00, 6985.71it/s]


<a id='step2'></a>
## Step 2: Train the Model

As training of this type of the Neural network can take a long time it might be useful to save the state of the weights and resume training in the later time. It is possible to do this by using lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training. However, this will not tell us if the model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models. For more details about minimising overfitting in image captioning please see  section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('saved_models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('saved_models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

print("\n=== Done ===")

Epoch [1/3], Step [100/4314], Loss: 3.5376, Perplexity: 34.3841
Epoch [1/3], Step [200/4314], Loss: 3.3402, Perplexity: 28.22587
Epoch [1/3], Step [300/4314], Loss: 3.1596, Perplexity: 23.5619
Epoch [1/3], Step [400/4314], Loss: 3.2330, Perplexity: 25.3544
Epoch [1/3], Step [500/4314], Loss: 2.8569, Perplexity: 17.4083
Epoch [1/3], Step [600/4314], Loss: 2.5816, Perplexity: 13.2189
Epoch [1/3], Step [700/4314], Loss: 2.7608, Perplexity: 15.8121
Epoch [1/3], Step [800/4314], Loss: 2.5825, Perplexity: 13.2299
Epoch [1/3], Step [900/4314], Loss: 2.6154, Perplexity: 13.6733
Epoch [1/3], Step [1000/4314], Loss: 2.6329, Perplexity: 13.9144
Epoch [1/3], Step [1100/4314], Loss: 2.3588, Perplexity: 10.5778
Epoch [1/3], Step [1200/4314], Loss: 2.5928, Perplexity: 13.3673
Epoch [1/3], Step [1300/4314], Loss: 2.4594, Perplexity: 11.6974
Epoch [1/3], Step [1400/4314], Loss: 2.7120, Perplexity: 15.0601
Epoch [1/3], Step [1500/4314], Loss: 2.1967, Perplexity: 8.99556
Epoch [1/3], Step [1600/4314], Lo

<a id='step3'></a>
## Step 3: Validate the Model

The suggested approach to validating the model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images. Then, we can use [this](https://github.com/tylin/coco-caption) script to calculate the BLEU score of your model.  
- You can read more about the **BLEU** score, along with other evaluation metrics (such as **TEOR** and **Cider**) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).
- For more information about how to use the **annotation file**, check out the [website](http://cocodataset.org/#download) for the COCO dataset.